In [ ]:
import folium
import pandas as pd
import altair as alt
import geopandas as gpd
from datetime import datetime, timedelta
import branca

In [ ]:
county_data_time_series =  pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

county_data_time_series_tx = county_data_time_series[county_data_time_series['Province_State']=='Texas']

In [ ]:
today = datetime.today()

dates = [today - timedelta(days=7),
        today - timedelta(days=6), 
        today - timedelta(days=5), 
        today - timedelta(days=4),
        today - timedelta(days=3),
        today - timedelta(days=2),
        today - timedelta(days=1)]

for item in range(len(dates)): 
    dates[item] = dates[item].strftime('%#m/%#d/%y')

In [ ]:
columns = ['Admin2']

for item in range(len(dates)):
    columns.append(dates[item])

In [ ]:
county_data = county_data_time_series_tx[columns]

In [ ]:
county_data.head()

In [ ]:
county_time = pd.melt(county_data, id_vars=['Admin2'])

county_time.rename(columns={'variable':'Date', 'value':'Confirmed Cases'}, inplace=True)

county_time.head()

In [ ]:
texas_counties = gpd.read_file('https://opendata.arcgis.com/datasets/9b2eb7d232584572ad53bad41c76b04d_0.geojson')

In [ ]:
texas_counties = texas_counties[['CNTY_NM',
                                 'Shape__Area',
                                 'Shape__Length',
                                 'geometry']]

#texas_counties['Graph'] = ""

In [ ]:
#this was for a section of code I could not get to work.  
#I'm still working out a for loop that will draw each polygon seperately, 
#then add a popup map that will show the cumulative cases over the last seven days

'''
for county in texas_counties['CNTY_NM']:
    chart = alt.Chart(county_time[county_time['Admin2']==county]
                     ).mark_bar().encode(x=alt.X('Date', sort = dates),
                                         y=alt.Y('Confirmed Cases', 
                                                 axis=alt.Axis(title="Cumulative Confirmed Cases")),                                                                                
                                         color=alt.value('red')
                     ).properties(
                           title = county
                     )
    
    vis = chart.to_json()
    
    texas_counties.loc[texas_counties['CNTY_NM']==county, 'Graph']=vis
'''

In [ ]:
'''
#example of what the charts would look like
chart

'''

In [ ]:
texas_counties = texas_counties.merge(county_data, left_on='CNTY_NM', right_on='Admin2')

In [ ]:
columns = ['CNTY_NM', 'Shape__Area', 'Shape__Length', 'geometry']

columns.append(dates[6])

texas_counties = texas_counties[columns]

In [ ]:
texas_counties.rename(columns={dates[6]:'Cases'}, inplace=True)

In [ ]:
county_pop = pd.read_csv('https://demographics.texas.gov/Resources/TPEPP/Estimates/2018/2018_txpopest_county.csv')

county_pop.head()

In [ ]:
county_pop = county_pop[['county', 'july1_2018_pop_est']]

county_pop.rename(columns={'july1_2018_pop_est':'Pop'}, inplace=True)

texas_counties = texas_counties.merge(county_pop, left_on='CNTY_NM', right_on='county')

In [ ]:
texas_counties['CasesX'] = round((texas_counties['Cases']/texas_counties['Pop'])*100000, 2)

In [ ]:
texas_counties.drop(['county'], axis=1, inplace=True)

texas_counties.head()

In [ ]:
colormap = branca.colormap.LinearColormap(vmin=0, 
                                          vmax=1000, 
                                          colors=['lightyellow','yellow','orange','red','darkred'],
                                          caption="Cumulative Confirmed Cases")

In [ ]:
m = folium.Map(location = [31.4, -99.9018], 
               tiles = 'Stamen Toner',
               zoom_start = 6)

tx_county_json = texas_counties.to_json()

folium.GeoJson(data=tx_county_json,
               style_function=lambda x: {
                    "fillColor": colormap(x['properties']['Cases'])
                    if x['properties']['Cases'] is not 0
                    else "Transparent",
                    "color": "black",
                    "fillOpacity": 0.7,
                },
               tooltip=folium.features.GeoJsonTooltip(fields=['CNTY_NM' , 'Cases', 'Pop', 'CasesX'],
                                                      aliases=['County:', 'Cases:', 'Pop.:', 'Cases/100K:']),
               name='Total Confirmed Cases').add_to(m)

folium.LayerControl().add_to(m)

colormap.add_to(m)

m

In [ ]:
colormap = branca.colormap.LinearColormap(vmin=texas_counties['CasesX'].quantile(0), 
                                          vmax=texas_counties['CasesX'].quantile(1), 
                                          colors=['lightyellow','yellow','orange','red','darkred'],
                                          caption="Cases per 100,000")

In [ ]:
m1 = folium.Map(location = [31.4, -99.9018], 
               tiles = 'Stamen Toner',
               zoom_start = 6)

tx_county_json = texas_counties.to_json()

folium.GeoJson(data=tx_county_json,
               style_function=lambda x: {
                    "fillColor": colormap(x['properties']['CasesX'])
                    if x['properties']['CasesX'] is not 0
                    else "Transparent",
                    "color": "black",
                    "fillOpacity": 0.7,
                },
               tooltip=folium.features.GeoJsonTooltip(fields=['CNTY_NM' , 'Cases', 'Pop', 'CasesX'],
                                                      aliases=['County:', 'Cases:', 'Pop.:', 'Cases/100K:'])).add_to(m1)

colormap.add_to(m1)

m1